In [16]:
import requests
import csv
import time
import random
import os
import glob
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
from tqdm import tqdm

In [15]:
# ------------------------------------------------------------------------------
# [설정] 헤더 및 URL
# ------------------------------------------------------------------------------
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

# 매거진(알아두면 좋은 꿀팁) 리스트 URL (페이지 번호는 뒤에 붙임)
magazine_list_url = "https://www.10000recipe.com/issue/view.html?cid=10000know&types=magazine&page="
base_domain = "https://www.10000recipe.com"

# 저장할 CSV 헤더 (팁/매거진에 맞는 항목으로 구성)
fieldnames = [
    'Link', 'Title', 'Main_Thumbnail', 'Intro_Summary', 'Content_Steps'
]

def main():
    # ------------------------------------------------------------------------------
    # [1] 파일 생성 및 이어하기(Resume) 설정
    # ------------------------------------------------------------------------------
    print("=" * 60)
    print(" [만개의 레시피 - 매거진/꿀팁 크롤러] 시작")
    print("=" * 60)

    # 기존 파일 확인
    existing_files = sorted(glob.glob("10000recipe_magazine_*.csv"), key=os.path.getmtime, reverse=True)
    filename = ""
    processed_links = set()

    if existing_files:
        latest_file = existing_files[0]
        print(f"\n[알림] 최근 작업 파일을 발견했습니다: '{latest_file}'")
        resume = input("  -> 이 파일에 이어서 작업하시겠습니까? (y/n): ").strip().lower()
        if resume == 'y':
            filename = latest_file
            try:
                with open(filename, 'r', encoding='utf-8-sig') as f:
                    reader = csv.DictReader(f)
                    for row in reader:
                        if 'Link' in row and row['Link']:
                            processed_links.add(row['Link'])
                print(f"  -> ✅ 이미 수집된 {len(processed_links)}개의 글을 건너뜁니다.")
            except:
                filename = ""

    if not filename:
        filename = f"10000recipe_magazine_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        with open(filename, 'w', newline='', encoding='utf-8-sig') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
        print(f"\n📁 새로운 파일 생성: '{filename}'")

    # ------------------------------------------------------------------------------
    # [2] 크롤링 루프 (페이지 순회)
    # ------------------------------------------------------------------------------
    # 원하는 페이지 범위를 설정하세요 (예: 1페이지부터 20페이지까지)
    target_start_page = 1
    target_end_page = 1

    print(f"\n>>> 크롤링 시작 (페이지 {target_start_page} ~ {target_end_page})")

    for page_no in range(target_start_page, target_end_page + 1):
        try:
            print(f"\n--- [Page {page_no}] 목록 수집 중 ---")

            # 1. 목록 페이지 접속
            response = requests.get(magazine_list_url + str(page_no), headers=headers, timeout=10)
            if response.status_code != 200:
                print(f"페이지 접속 실패: {response.status_code}")
                continue

            soup = BeautifulSoup(response.text, 'html.parser')

            # 2. 목록에서 아이템 추출 (이전 대화에서 확인한 구조: .theme_list .thumbnail)
            items = soup.select('.theme_list .thumbnail')

            if not items:
                print("더 이상 게시물이 없습니다. 종료합니다.")
                break

            # 3. 각 아이템별 상세 페이지 크롤링
            for item in tqdm(items, desc=f"Page {page_no} 상세 수집"):
                try:
                    # 링크 추출
                    href = item.get('href')
                    if not href: continue

                    full_link = base_domain + href

                    # 이미 수집한 링크면 패스
                    if full_link in processed_links:
                        continue

                    # 상세 페이지 접속
                    detail_res = requests.get(full_link, headers=headers, timeout=10)
                    if detail_res.status_code != 200: continue

                    d_soup = BeautifulSoup(detail_res.text, 'html.parser')

                    # --- 데이터 추출 (매거진/팁 구조) ---

                    # (1) 제목
                    # 상세 페이지 내부의 제목 태그 (보통 <h3> 또는 .view2_summary_st3)
                    # 만약 구조가 다르면 목록에서 가져온 제목을 써도 되지만, 상세페이지 제목이 정확함
                    title_tag = d_soup.select_one('.view2_summary h3')
                    title = title_tag.get_text(strip=True) if title_tag else "제목 없음"

                    # (2) 메인 이미지
                    main_img_tag = d_soup.select_one('#main_thumbs')
                    main_img = main_img_tag['src'] if main_img_tag else "None"

                    # (3) 요약(인트로) 글
                    intro_tag = d_soup.select_one('#recipeIntro')
                    intro = intro_tag.get_text(strip=True) if intro_tag else "None"

                    # (4) 본문 내용 (Steps)
                    # 팁도 레시피처럼 view_step_cont 구조를 사용하는 경우가 많음
                    step_divs = d_soup.select('div.view_step_cont')
                    step_list = []

                    if step_divs:
                        for i, step_div in enumerate(step_divs):
                            # 텍스트
                            desc_tag = step_div.select_one('div.media-body')
                            desc = desc_tag.get_text(strip=True) if desc_tag else ""
                            # 이미지
                            s_img_tag = step_div.select_one('img')
                            s_img = s_img_tag['src'] if s_img_tag else ""

                            step_list.append(f"[{i+1}] {desc} || {s_img}")
                    else:
                        # step 구조가 없는 통이미지/통글 형태일 경우 (예외처리)
                        # view2_view_cont 내의 텍스트를 긁어옴
                        body_content = d_soup.select_one('.view2_view_cont')
                        if body_content:
                            step_list.append(body_content.get_text(strip=True))

                    content_text = "\n".join(step_list)

                    # --- CSV 저장 ---
                    row_data = {
                        'Link': full_link,
                        'Title': title,
                        'Main_Thumbnail': main_img,
                        'Intro_Summary': intro,
                        'Content_Steps': content_text
                    }

                    with open(filename, 'a', newline='', encoding='utf-8-sig') as csvfile:
                        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                        writer.writerow(row_data)

                    processed_links.add(full_link)

                    # 차단 방지를 위한 대기
                    time.sleep(random.uniform(0.5, 1.0))

                except Exception as e:
                    print(f"  [Error] 상세 페이지 처리 중 오류: {e}")
                    continue

        except Exception as e:
            print(f"[Page Error] 페이지 {page_no} 처리 중 오류: {e}")

    print(f"\n🎉 크롤링 완료! 데이터가 '{filename}'에 저장되었습니다.")

if __name__ == "__main__":
    main()

 [만개의 레시피 - 매거진/꿀팁 크롤러] 시작

📁 새로운 파일 생성: '10000recipe_magazine_20260108_160906.csv'

>>> 크롤링 시작 (페이지 1 ~ 1)

--- [Page 1] 목록 수집 중 ---


Page 1 상세 수집: 100%|██████████| 20/20 [00:25<00:00,  1.28s/it]


🎉 크롤링 완료! 데이터가 '10000recipe_magazine_20260108_160906.csv'에 저장되었습니다.


In [18]:
df = pd.read_csv('10000recipe_magazine_20260108_160906.csv')
df

,Link,Title,Main_Thumbnail,Intro_Summary,Content_Steps
0,https://www.10000recipe.com/recipe/7024170,슈퍼푸드 아보카도 손질과 보관,https://recipe1.ezmember.co.kr/cache/recipe/20...,슈퍼 푸드로 알려져 있는 아보카도 ! 맛도 좋고 건강에도 좋은 아보카도 간단하게 소...,"[1] 남미에서 재배 멕시코에서 70% 재배되고 아보카도는 눈 건강, 항암 효과, ..."
1,https://www.10000recipe.com/recipe/7023971,브로콜리 손질법/보관법/조리법,https://recipe1.ezmember.co.kr/cache/recipe/20...,"브로콜리는 강력한 항암성분인 설포라판이 함유되어있고 해독, 염증, 피부, 눈건강, ...",[1] 고르는법 - 둥굴고 진한 녹색이 좋은 브로콜리이고 노래지고 봉우리가 펴 있다...
2,https://www.10000recipe.com/recipe/6997356,라면 맛있게 끓이기 2탄,https://recipe1.ezmember.co.kr/cache/recipe/20...,라면 맛있게 만들기 제 2탄!이번엔 여러분이 추천해주신 레시피로!어떤 재료를 추천해...,[1] 라면 + 설탕스프와 함께 설탕 반스푼 넣어주면 분식집 맛 그대로!! || h...
3,https://www.10000recipe.com/recipe/6996955,라면 맛있게 끓이기 1탄,https://recipe1.ezmember.co.kr/cache/recipe/20...,XX을 넣으면 라면이 맛있어진다?일단 보고 따라해보세요!후회안하실걸요?,[1] 라면 + 식초라면이 다 끓었을 때쯤 식초 1~2숟가락을 넣어요.식초를 넣으면...
4,https://www.10000recipe.com/recipe/6996797,양파 보관법,https://recipe1.ezmember.co.kr/cache/recipe/20...,여기도 쓰고 저기도 쓰는 양파어떻게 보관하는게 좋을까?잘못보관하면 금방 썩어버려 아...,[1] 뿌리쪽을 잘라내지 않고 껍질을 벗겨요. || https://recipe1.e...
5,https://www.10000recipe.com/recipe/6996785,동그랑땡 쉽게 만들기,https://recipe1.ezmember.co.kr/cache/recipe/20...,명절마다 먹는 동그랑땡누구나 좋아해 많은 사랑을 받는 요리인데요.평소에도 먹으면 좋...,[1] 모든 재료를 볼에 넣고 섞어 반죽을 만들어요.(채소는 다지고 두부는 물기를 ...
6,https://www.10000recipe.com/recipe/6996207,물에 씻으면 오히려 안좋은 재료들,https://recipe1.ezmember.co.kr/cache/recipe/20...,씻으면 안되는 재료가 있다는 것 여러분은 알고 계셨나요? 우리가 자주 먹는 재료 중...,[1] 요리 재료를 구입했다면 깨끗하게 세척하는 것이 필수 과정입니다. 하지만 물에...
7,https://www.10000recipe.com/recipe/6996166,베이킹소다 활용법,https://recipe1.ezmember.co.kr/cache/recipe/20...,베이킹소다 어떻게 활용하면 좋을까~? 만개가 베이킹소다 활용법 알려드려요! 지금 따...,[1] 첫째 ~ 베이킹 소다는 스텐 조리도구 세척에 좋아요. || https://r...
8,https://www.10000recipe.com/recipe/6996052,생라면 먹고 남은 건더기 스프 활용 법,https://recipe1.ezmember.co.kr/cache/recipe/20...,생라면 먹고 남은 건더기 스프 활용하기! 남은 건더기 스프는 활용하기 어려워 버리는...,"[1] 생라면, 라면스프만 빼먹고 남은 후레이크 언젠간 먹어야지 하고 부엌 선반이나..."
9,https://www.10000recipe.com/recipe/6996047,과일껍질 활용법,https://recipe1.ezmember.co.kr/cache/recipe/20...,"먹고 남은 과일껍질 어디까지 써봤니?사과, 레몬, 오렌지 등 먹고 남은 과일껍질을어...",[1] 사과 - 사과 껍질에는 '트리테르페놀' 이라는 산성 성분이 함유되어 탄 냄비...
